In [1]:
# Clean runtime state
!rm -rf /content/results
!rm -rf /content/sarcasm_model


In [3]:
from google.colab import drive
import os

if not os.path.exists("/content/drive"):
    drive.mount("/content/drive")
else:
    print("Drive already mounted")


Mounted at /content/drive


In [4]:
!pip install -q transformers datasets accelerate evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.5 MB/s eta 0:00:00


In [5]:
ABBREVIATIONS = {
    "idk": "i do not know",
    "imo": "in my opinion",
    "smh": "shaking my head",
    "lol": "laughing out loud",
    "btw": "by the way",
    "brb": "be right back",
}

def expand_abbreviations(text):
    words = text.lower().split()
    return " ".join([ABBREVIATIONS.get(w, w) for w in words])


In [6]:
!pip install kaggle


In [7]:
import kagglehub
import os
import pandas as pd

path = kagglehub.dataset_download(
    "farhanch67/twitter-sracastic-classification-dataset"
)

csv_path = os.path.join(path, "Sarcasm.csv")
df_sarcasm = pd.read_csv(csv_path)

df_sarcasm.head()



Using Colab cache for faster access to the 'twitter-sracastic-classification-dataset' dataset.


,Unnamed: 0,tweet,sarcastic,sarcasm,irony,satire,understatement,overstatement,rhetorical_question
0,0,The only thing I got from college is a caffein...,1,0.0,1.0,0.0,0.0,0.0,0.0
1,1,I love it when professors draw a big question ...,1,1.0,0.0,0.0,0.0,0.0,0.0
2,2,Remember the hundred emails from companies whe...,1,0.0,1.0,0.0,0.0,0.0,0.0
3,3,Today my pop-pop told me I was not “forced” to...,1,1.0,0.0,0.0,0.0,0.0,0.0
4,4,@VolphanCarol @littlewhitty @mysticalmanatee I...,1,1.0,0.0,0.0,0.0,0.0,0.0


In [9]:
df_sarcasm.columns


Index(['Unnamed: 0', 'tweet', 'sarcastic', 'sarcasm', 'irony', 'satire',
       'understatement', 'overstatement', 'rhetorical_question'],
      dtype='object')

In [8]:
import pandas as pd

df = df_sarcasm.copy()

df = df[["tweet", "sarcastic"]]
df.columns = ["text", "label"]

df["label"] = df["label"].astype(int)


In [10]:
df["text"] = df["text"].astype(str).apply(expand_abbreviations)


In [11]:
from datasets import Dataset

sarcasm_dataset = Dataset.from_pandas(df)
sarcasm_dataset = sarcasm_dataset.train_test_split(test_size=0.2, seed=42)


In [12]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize_fn(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

tokenized_sarcasm = sarcasm_dataset.map(tokenize_fn, batched=True)
tokenized_sarcasm = tokenized_sarcasm.remove_columns(["text"])
tokenized_sarcasm.set_format("torch")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/2774 [00:00<?, ? examples/s]

Map:   0%|          | 0/694 [00:00<?, ? examples/s]

In [13]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2,
    id2label={0: "not_sarcastic", 1: "sarcastic"},
    label2id={"not_sarcastic": 0, "sarcastic": 1}
)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_dir="./logs",
    save_total_limit=2,
)



In [15]:
from datasets import Dataset
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(
    df,
    test_size=0.15,
    stratify=df["label"],
    random_state=42
)
train_dataset = Dataset.from_pandas(train_df.reset_index(drop=True))
val_dataset = Dataset.from_pandas(val_df.reset_index(drop=True))


In [16]:
def tokenize_function(batch):
    return tokenizer(
        batch["text"],
        padding="max_length",
        truncation=True,
        max_length=128
    )


In [17]:
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)
train_dataset = train_dataset.remove_columns(
    [col for col in train_dataset.column_names if col not in ["input_ids", "attention_mask", "label"]]
)

val_dataset = val_dataset.remove_columns(
    [col for col in val_dataset.column_names if col not in ["input_ids", "attention_mask", "label"]]
)

train_dataset.set_format("torch")
val_dataset.set_format("torch")


Map:   0%|          | 0/2947 [00:00<?, ? examples/s]

Map:   0%|          | 0/521 [00:00<?, ? examples/s]

In [18]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

trainer.train()


/tmp/ipython-input-3027770107.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:wandb: WARNING Invalid choice
wandb: Enter your choice:

 3


wandb: You chose "Don't visualize my results"


Step,Training Loss
500,0.499000


TrainOutput(global_step=555, training_loss=0.4906808732866167, metrics={'train_runtime': 218.5334, 'train_samples_per_second': 40.456, 'train_steps_per_second': 2.54, 'total_flos': 292786067879424.0, 'train_loss': 0.4906808732866167, 'epoch': 3.0})

In [19]:
SAVE_PATH = "/content/drive/MyDrive/sarcasm_model"

trainer.save_model(SAVE_PATH)
tokenizer.save_pretrained(SAVE_PATH)


('/content/drive/MyDrive/sarcasm_model/tokenizer_config.json',
 '/content/drive/MyDrive/sarcasm_model/special_tokens_map.json',
 '/content/drive/MyDrive/sarcasm_model/vocab.txt',
 '/content/drive/MyDrive/sarcasm_model/added_tokens.json',
 '/content/drive/MyDrive/sarcasm_model/tokenizer.json')

In [20]:
!ls /content/drive/MyDrive/sarcasm_model


config.json	   special_tokens_map.json  tokenizer.json     vocab.txt
model.safetensors  tokenizer_config.json    training_args.bin


In [21]:
from transformers import pipeline
import torch

device = 0 if torch.cuda.is_available() else -1

sarcasm_pipeline = pipeline(
    "text-classification",
    model="/content/drive/MyDrive/sarcasm_model",
    tokenizer="/content/drive/MyDrive/sarcasm_model",
    device=device
)

sarcasm_pipeline("Oh great, I just love waiting in traffic.")


Device set to use cuda:0


[{'label': 'sarcastic', 'score': 0.7888455390930176}]